# Projet

2241.3 Python, HE-ARC, 2017, Yoan Blanc

## Objectifs

* réaliser un projet Python fun
* se familiariser avec l'écosystème du développeur Python (PyPI, virtualenv)
* respecter les bonnes pratiques en matières de documentations (sphinx) ou tests (py.test)
* vendre son projet

## Détails

Ce projet va compter pour **25% de la note finale** (code et présentation comptent).

Des bonus (en points ou en nature) seront possibles pour différentes catégories comme : qualité, marketing ou originalité. Il est évidemment possible de cumuler les récompenses bonus.

![Hello?](http://i1-news.softpedia-static.com/images/news-700/Grammys-2014-Adele-Wins-Her-10th-Grammy-Tweets-Regret-for-Not-Being-Here.jpg)

## Cahier des charges

Réaliser un [bot pour Discord](https://discordapp.com/developers/docs/intro) pour réaliser l'idée de votre choix. À vous de trouver une idée **simple**, _réaliste_ et fun. C'est plus motivant quand c'est fun.

![Twitter Bot](http://imgs.xkcd.com/comics/twitter_bot.png)

### À rendre

Pour le **dimanche 4 juin 2017, à 23:59:59.99**: un paquet téléchargeable via `pip` (PyPI ou Github).

1. Votre projet sera open source et aura un dépôt public (_Github_, _Gitlab_, _Bitbucket_, etc.)
2. Il sera possible de l'installer via une  simple commande `pip`. Voir: [Structuring Your Project](http://docs.python-guide.org/en/latest/writing/structure/)
3. [Read the docs](https://readthedocs.org/) est un plateforme d'hébergement de documentation. Accessoirement, votre projet pourra s'y trouver. Voir: [Project Documentation](http://docs.python-guide.org/en/latest/writing/documentation/)
4. Accessoirement, Il sera possible d'exécuter une batterie de tests afin de valider que tout va bien. Voir [Testing your code](http://docs.python-guide.org/en/latest/writing/tests/)
5. Accessoirement, votre projet sera soumis à de l'intégration continue (e.g. via [Travis](http://travis-ci.org/)).

Accessoirement = bonus.

### Création des équipes

Formez des groupes de deux personnes maximum. Voire de une personne pour les électrons libres. Pinguez-moi l'URL de votre dépôt public, le nom des membres et décrivez votre idée dans le `README.rst` à la racine du projet.

## Alternative

L'exemple ci-dessous est assez bas niveau. Si vous préférez vous concentrer sur la partie _bot_ plutôt que la partie _Web Socket_, je suis partant de vous laisser utiliser l'[API Python officielle](http://discordpy.readthedocs.io/en/latest/). Elle masque une partie de la complexité.

En contrepartie, je serai plus exigeant sur la qualité du projet d'un point de vue de l'expérience utilisateur.

# Bot Discord 101

Version adaptée de l'article en anglaise suivant: [A Discord Bot with asyncio](https://tutorials.botsfloor.com/a-discord-bot-with-asyncio-359a2c99e256).

Le bot ci-dessous requiert de créer une application _Discord_, de la promouvoir en tant que Bot puis de l'ajouter à un serveur avec l'URL ci-dessous.

```
https://discordapp.com/oauth2/authorize?scope=bot&permissions=o&client_id=CLIENT_ID
```

Le jeton du bot permet l'identification. Il est important d'installer une version récente d'aiohttp (`>=2.0.0`).

In [4]:
! pip install --user --upgrade aiohttp

Requirement already up-to-date: aiohttp in c:\users\gabriel.griesser\appdata\roaming\python\python36\site-packages
Requirement already up-to-date: yarl<0.11,>=0.10.0 in c:\users\gabriel.griesser\appdata\roaming\python\python36\site-packages (from aiohttp)
Requirement already up-to-date: chardet in c:\users\gabriel.griesser\appdata\roaming\python\python36\site-packages (from aiohttp)
Requirement already up-to-date: async-timeout>=1.2.0 in c:\users\gabriel.griesser\appdata\roaming\python\python36\site-packages (from aiohttp)
Requirement already up-to-date: multidict>=2.1.4 in c:\users\gabriel.griesser\appdata\roaming\python\python36\site-packages (from aiohttp)


## Fonctionnement dans les grandes lignes

Voici les étapes de connexion d'un bot.

1. On demande l'adresse de la _Web Socket_ à l'API
2. On se connecte à la _Web Socket_ puis s'y authentifie (2 Identify)
3. Ensuite, une tâche de fond envoie des _heartbeats_ à intervales réguliers
4. À partir de là :
   - les évènements arrivent par la _Web Socket_
   - les actions du bot sont faites par l'API REST (_HTTP_)

In [1]:
"""Bot exemple qui répond à @greut."""

import asyncio
import json
import zlib

import aiohttp

# Jupyter hack pour recréer une boucle.
# Pas nécessaire hors de Jupyter
loop = asyncio.get_event_loop()
if loop.is_closed():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
# fin du hack

TOKEN = ...

URL = "https://discordapp.com/api"
HEADERS = {
    "Authorization": f"Bot {TOKEN}",
    "User-Agent": "DiscordBot (http://he-arc.ch/, 0.1)"
}

async def api_call(path, method="GET", **kwargs):
    """Effectue une  requête sur l'API REST de Discord."""
    default = {"headers": HEADERS}
    kwargs = dict(default, **kwargs)
    with aiohttp.ClientSession() as session:
        async with session.request(method, f"{URL}{path}", **kwargs) as response:
            if 200 == response.status:
                return await response.json()
            elif 204 == response.status:
                return {}
            else:
                body = await response.text()
                raise AssertionError(f"{response.status} {response.reason} was unexpected.\n{body}")   

async def send_message(recipient_id, content):
    """Envoie un message à l'utilisateur donné."""
    channel = await api_call("/users/@me/channels", "POST", json={"recipient_id": recipient_id})
    return await api_call(f"/channels/{channel['id']}/messages", "POST", json={"content": content})

# Pas très joli, mais ça le fait.
last_sequence = None

async def heartbeat(ws, interval):
    """Tâche qui informe Discord de notre présence."""
    while True:
        await asyncio.sleep(interval / 1000)
        print("> Heartbeat")
        await ws.send_json({'op': 1,  # Heartbeat
                            'd': last_sequence})


async def identify(ws):
    """Tâche qui identifie le bot à la Web Socket (indispensable)."""
    await ws.send_json({'op': 2,  # Identify
                        'd': {'token': TOKEN,
                              'properties': {},
                              'compress': True,  # implique le bout de code lié à zlib, pas nécessaire.
                              'large_threshold': 250}})
        
async def start(ws):
    """Lance le bot sur l'adresse Web Socket donnée."""
    global last_sequence  # global est nécessaire pour modifier la variable
    with aiohttp.ClientSession() as session:
        async with session.ws_connect(f"{ws}?v=5&encoding=json") as ws:
            async for msg in ws:
                if msg.tp == aiohttp.WSMsgType.TEXT:
                    data = json.loads(msg.data)
                elif msg.tp == aiohttp.WSMsgType.BINARY:
                    data = json.loads(zlib.decompress(msg.data))
                else:
                    print("?", msg.tp)

                # https://discordapp.com/developers/docs/topics/gateway#gateway-op-codes
                if data['op'] == 10:  # Hello
                    asyncio.ensure_future(heartbeat(ws, data['d']['heartbeat_interval']))
                    await identify(ws)
                elif data['op'] == 11:  # Heartbeat ACK
                    print("< Heartbeat ACK")
                elif data['op'] == 0:  # Dispatch
                    last_sequence = data['s']
                    if data['t'] == "MESSAGE_CREATE":
                        print(data['d'])
                        if data['d']['author']['username'] == 'greut':
                            task = asyncio.ensure_future(send_message(data['d']['author']['id'],
                                                                      data['d']['content']))
                            
                            if data['d']['content'] == 'quit':
                                print('Bye bye!')
                                # On l'attend l'envoi du message ci-dessus.
                                await asyncio.wait([task])
                                break
                    else:
                        print('Todo?', data['t'])
                else:
                    print("Unknown?", data)


async def main():
    response = await api_call('/gateway')
    await start(response['url'])

    
# Lancer le programme.
loop = asyncio.get_event_loop()
loop.set_debug(True)
loop.run_until_complete(main())
loop.close()

ModuleNotFoundError: No module named 'aiohttp'

In [7]:
#TIC TAC TOE python
import discord
import random

client = discord.Client()

@client.event
async def on_message(message):
    #we do not want the bot to reply itself
    if message.author == client.user:
        return #qqch
    if message.content.startwith('$guess'):
        return #qqch

    
    
#TIC TAC TOE

def draw(b):
    #print the board
    print('   |   |')
    #Print the last line (7 to 9)
    print(' ' + b[7] + ' | ' + b[8] + ' | ' + b[9])
    print('   |   |')
    print('-----------')
    #Print the middle line (4 to 6)
    print('   |   |')
    print(' ' + b[4] + ' | ' + b[5] + ' | ' + b[6])
    print('   |   |')
    print('-----------')
    #Print the first line
    print('   |   |')
    print(' ' + b[1] + ' | ' + b[2] + ' | ' + b[3])
    print('   |   |')
    
    
def inputPlayer():
    #The player type a letter (O or X)
    #The function returns a list with the player's letter and computer's letter
    letter = ''
    while not(letter == 'X' or letter == 'O'):
        print('Quelle lettre souhaitez-vous être, O ou X ?')
        letter = input().upper()
        
    if letter == 'O':
        return['O', 'X']
    else:
        return['X', 'O']

def getMovePlayer(b):
    #Let the player type his move 
    position = ''
    while position not in '1 2 3 4 5 6 7 8 9'.split() or not isFree(b, int(move)):
        print('Dans quelle case voulez-vous placer votre lettre ?')
        position = input()
    return int(position)
    
    
def winnerIf(b):
    #b = board and l = letter.
    #Define all the ways to win
    #Return true if win
    board = 
    [
        None, 'O', None,
        None, 'O', 'X',
        'X',  'O', None
    ]
    return (
    (assert ['O', 'O', 'O'] == board[1::3]) or   #Middle column
    (assert [None, None, 'O'] == board[0::3]) or #Left column
    (assert [None, 'X', None] == board[2::3]) or #Right column
    (assert [None, 'O', None] == board[0::4]) or #diagonal leftRight
    #assert [None, 'O', 'X'] == board[2:-2:2]    #diagonal rightLeft
    (assert [None, 'O', 'X'] == board[2::2])  or #diagonal rightLeft
    (assert [None, 'O', 'X'] == board[3:5:])   or #middle Line
    (assert [None, 'O', None] == board[0:2:])  or #first Line
    (assert ['X', 'O', None] == board[6:8:]       #last Line
    ))
    

def firstPlayer():
    # Who begin ?
    if(random.randint(0,1) == 0):
        return 'player'
    else:
        return 'computer'

def replay():
    # Return false if the player doesn't want to play again, true otherwise
    print('Voulez-vous rejouer ? (oui, non)')
    if(input().lower().startswith('o')):
        return True
    else:
        return False
    
def placeLetter(board, letter, position):
    #Place the letter in the case
    board[position] = letter
    
def isFree(b, position):
    #Return true if the case at the position is free
    return b[position] == ' '

ModuleNotFoundError: No module named 'discord'